# About Data Set

This is the Olist Dataset that I picked up from Kaggle. I will first conduct data wrangling methodoligies and find if the data looks good and merge the required the datasets before I start exploring the dataset.

# Data Wrangling

## Customer_Data

In [33]:
#importing required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [19]:
#To check number of unique values in Customer dataset
df_customer=pd.read_csv('olist_customers_dataset.csv')
df_customer.head()
df_customer.agg(["nunique", "count"]).T

,nunique,count
customer_id,99441,99441
customer_unique_id,96096,99441
customer_zip_code_prefix,14994,99441
customer_city,4119,99441
customer_state,27,99441


In [82]:
#To check for null values
df_customer.isnull().sum().sum()

0

It looks like we have 96096 unique customer with around 3345 repeat purchases.

## Geolocation Data

In [57]:
#To check number of unique values in Geolocation dataset
df_geolocation=pd.read_csv('olist_geolocation_dataset.csv')
df_geolocation.head()
df_geolocation.agg(["nunique", "count"]).T

,nunique,count
geolocation_zip_code_prefix,19015,1000163
geolocation_lat,717358,1000163
geolocation_lng,717613,1000163
geolocation_city,8011,1000163
geolocation_state,27,1000163


In [80]:
#to check for null values
df_geolocation.isnull().sum().sum()

0

## Orders Data

In [55]:

df_orders=pd.read_csv('olist_orders_dataset.csv')
df_orders.head()
#converting all dates to datetime objects
df_orders["order_delivered_customer_date"]=pd.to_datetime(df_orders["order_delivered_customer_date"])
df_orders["order_estimated_delivery_date"]=pd.to_datetime(df_orders["order_estimated_delivery_date"])
df_orders["order_purchase_timestamp"]=pd.to_datetime(df_orders["order_purchase_timestamp"])
df_orders["order_approved_at"]=pd.to_datetime(df_orders["order_approved_at"])
df_orders["order_delivered_carrier_date"]=pd.to_datetime(df_orders["order_delivered_carrier_date"])
#To check number of unique values in Orders dataset
df_orders.agg(["nunique", "count"]).T


,nunique,count
order_id,99441,99441
customer_id,99441,99441
order_status,8,99441
order_purchase_timestamp,98875,99441
order_approved_at,90733,99281
order_delivered_carrier_date,81018,97658
order_delivered_customer_date,95664,96476
order_estimated_delivery_date,459,99441


# MOVE BELOW CODE TO DATA EXPLORATION

In [61]:
# finding difference between actual delivery date and estimated delivery date
df_orders['est_actual_delivery_diff']=df_orders['order_estimated_delivery_date'].dt.date-df_orders['order_delivered_customer_date'].dt.date
#finding difference between ordered date and delivery date
df_orders['delivery_time']=df_orders['order_delivered_customer_date'].dt.date-df_orders['order_purchase_timestamp'].dt.date
#finding difference between order approved and order handed to delivery agent
df_orders['olist_to_carrier_time']=df_orders['order_delivered_carrier_date'].dt.date-df_orders['order_approved_at'].dt.date
#finding taken for carrier to deliver items
df_orders['carrier_delivery_time']=df_orders['order_delivered_customer_date'].dt.date-df_orders['order_delivered_carrier_date'].dt.date

In [64]:
#Check the different types of order status
df_orders.groupby(by=['order_status']).count()

,order_id,customer_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,est_actual_delivery_diff,delivery_time,olist_to_carrier_time,carrier_delivery_time
order_status,,,,,,,,,,,
approved,2,2,2,2,0,0,2,0,0,0,0
canceled,625,625,625,484,75,6,625,6,6,75,6
created,5,5,5,0,0,0,5,0,0,0,0
delivered,96478,96478,96478,96464,96476,96470,96478,96470,96470,96462,96469
invoiced,314,314,314,314,0,0,314,0,0,0,0
processing,301,301,301,301,0,0,301,0,0,0,0
shipped,1107,1107,1107,1107,1107,0,1107,0,0,1107,0
unavailable,609,609,609,609,0,0,609,0,0,0,0


As we can see in the data 97% of the orders have been delivered and 0.6 percent of orders are cancelled and the rest 2.4% of the orders are in other status.As we want to understand regarding the delivered items, we shall only consider the orders that have been delivered

In [66]:
df_orders=df_orders[df_orders['order_status']=='delivered']
df_orders.agg(["nunique", "count"]).T

,nunique,count
order_id,96478,96478
customer_id,96478,96478
order_status,1,96478
order_purchase_timestamp,95956,96478
order_approved_at,88274,96464
order_delivered_carrier_date,80106,96476
order_delivered_customer_date,95658,96470
order_estimated_delivery_date,445,96478
est_actual_delivery_diff,198,96470
delivery_time,144,96470


In [75]:
#check for number of null values
df_orders.isnull().sum().sum()

65

As the number of null values are very small, we will remove the rows that contain them

In [79]:
df_orders.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
df_orders.agg(["nunique", "count"]).T

,nunique,count
order_id,96455,96455
customer_id,96455,96455
order_status,1,96455
order_purchase_timestamp,95933,96455
order_approved_at,88266,96455
order_delivered_carrier_date,80094,96455
order_delivered_customer_date,95643,96455
order_estimated_delivery_date,445,96455
est_actual_delivery_diff,198,96455
delivery_time,144,96455


## Order Items Data

In [83]:
#To check number of unique values in Order Items dataset
df_order_items=pd.read_csv('olist_order_items_dataset.csv')
df_order_items.head()
df_order_items.agg(["nunique", "count"]).T

,nunique,count
order_id,98666,112650
order_item_id,21,112650
product_id,32951,112650
seller_id,3095,112650
shipping_limit_date,93318,112650
price,5968,112650
freight_value,6999,112650


In [84]:
#check for number of null values
df_order_items.isnull().sum().sum()

0

In [85]:
df_order_items[df_order_items['order_id']=='8272b63d03f5f79c56e9e4120aec44ef']

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
57297,8272b63d03f5f79c56e9e4120aec44ef,1,270516a3f41dc035aa87d220228f844c,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57298,8272b63d03f5f79c56e9e4120aec44ef,2,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57299,8272b63d03f5f79c56e9e4120aec44ef,3,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57300,8272b63d03f5f79c56e9e4120aec44ef,4,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57301,8272b63d03f5f79c56e9e4120aec44ef,5,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57302,8272b63d03f5f79c56e9e4120aec44ef,6,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57303,8272b63d03f5f79c56e9e4120aec44ef,7,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57304,8272b63d03f5f79c56e9e4120aec44ef,8,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57305,8272b63d03f5f79c56e9e4120aec44ef,9,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57306,8272b63d03f5f79c56e9e4120aec44ef,10,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89


## Order Payments Data

In [86]:
#To check number of unique values in Order Payments dataset
df_order_payments=pd.read_csv('olist_order_payments_dataset.csv')
df_order_payments.head()
df_order_payments.agg(["nunique", "count"]).T

,nunique,count
order_id,99440,103886
payment_sequential,29,103886
payment_type,5,103886
payment_installments,24,103886
payment_value,29077,103886


In [87]:
#check for number of null values
df_order_payments.isnull().sum().sum()

0

## Order Reviews Data

In [89]:
#To check number of unique values in Order Reviews dataset
df_order_reviews=pd.read_csv('olist_order_reviews_dataset.csv')
df_order_reviews.head()
df_order_reviews.agg(["nunique", "count"]).T

,nunique,count
review_id,99173,100000
order_id,99441,100000
review_score,5,100000
review_comment_title,4600,11715
review_comment_message,36921,41753
review_creation_date,637,100000
review_answer_timestamp,99010,100000


In [99]:
#check for number of null values
df_order_reviews.isnull().sum().sum()

146532

The missing values is because many customers have not given any reviews for the items they have bought. As a result we shall just ignore the missing values for the time being

## Products Data

In [105]:
#To check number of unique values in Products dataset
df_products=pd.read_csv('olist_products_dataset.csv')
df_products.head()
df_products.agg(["nunique", "count"]).T

,nunique,count
product_id,32951,32951
product_category_name,73,32341
product_name_lenght,66,32341
product_description_lenght,2960,32341
product_photos_qty,19,32341
product_weight_g,2204,32949
product_length_cm,99,32949
product_height_cm,102,32949
product_width_cm,95,32949


In [111]:
#check for number of null values
df_products['product_category_name'].isnull().sum().sum()

610

As we can see, there are many missing values, we will remove the missing values if when we do an indepth product analysis. As of now we will let the missing values be.

 ## Category Name Translation Data

In [112]:
#To check number of unique values in Products dataset
df_prod_category_trans=pd.read_csv('product_category_name_translation.csv')
df_prod_category_trans.head()
df_prod_category_trans.agg(["nunique", "count"]).T

,nunique,count
product_category_name,71,71
product_category_name_english,71,71


In [120]:
# merging product catergory names in english to the products table
df_products=pd.merge(df_products,df_prod_category_trans,how='left',on='product_category_name')
df_products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english_x,product_category_name_english_y,product_category_name_english
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumery,perfumery,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,art,art,art
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,sports_leisure,sports_leisure,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0,baby,baby,baby
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0,housewares,housewares,housewares


## Sellers Data

In [121]:
#To check number of unique values in Products dataset
df_sellers=pd.read_csv('olist_sellers_dataset.csv')
df_sellers.head()
df_sellers.agg(["nunique", "count"]).T

,nunique,count
seller_id,3095,3095
seller_zip_code_prefix,2246,3095
seller_city,611,3095
seller_state,23,3095


In [123]:
#check for number of null values
df_sellers.isnull().sum().sum()

0